# SDM ETL code

Importeer pandas voor DataFrames en pyodbc voor database-connecties:

In [86]:
import sqlite3
import pandas as pd
import pyodbc
import numpy as np
import re


aenc_conn = sqlite3.connect("../../../Data/Raw/Sqlite/aenc.sqlite")
adventureworks_conn = sqlite3.connect("../../../Data/Raw/Sqlite/AdventureWorks.sqlite")

Database Connectie:

In [87]:
DB_SDM = {"servername": r"143.177.235.181", "database": "SDM_DEDS_P1", "username": "sa", "password": "Str0ngP@ssword"}
DB_NorthWind = {"servername": r"143.177.235.181", "database": "NorthWind", "username": "sa", "password": "Str0ngP@ssword"}
export_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_SDM['servername']};"
    f"DATABASE={DB_SDM['database']};"
    f"UID={DB_SDM['username']};"
    f"PWD={DB_SDM['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

NorthWind_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_NorthWind['servername']};"
    f"DATABASE={DB_NorthWind['database']};"
    f"UID={DB_NorthWind['username']};"
    f"PWD={DB_NorthWind['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

DataFrames maken voor tables:

In [88]:
def create_dataframes_sql(connection, db_type):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    if db_type == "sqlite":
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif db_type == "ssms":
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"

    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        try:
            dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
        except Exception as e:
            print(f"Error reading {table}: {e}")
            continue

    return dictionary


aenc = create_dataframes_sql(aenc_conn, "sqlite")
adventureworks = create_dataframes_sql(adventureworks_conn, "sqlite")
northwind = create_dataframes_sql(NorthWind_conn, "ssms")

#Errors en warnings zijn niet erg hier als het totaal 1 error en 4 UserWarnings zijn.

Error reading sysdiagrams: Could not decode to UTF-8 column 'definition' with text '������'


C:\Users\saadz\AppData\Local\Temp\ipykernel_4832\1853655394.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_4832\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_4832\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:\Us

Dictionaries met DataFrames mergen naar één Dictionary:

In [89]:
sdm = aenc | adventureworks | northwind #Alle drie mergen in één Dictionary met alle DataFrames

sdm['Products'] = sdm['Products'].rename(columns={
    "ProductID": "NWProductID",
    "ProductName": "Name", 
    "UnitPrice": "ListPrice"
})
sdm['Product'] = sdm['Product'].rename(columns={
    "id": "ProductID",
    "name": "Name",
    "prod_size": "Size",
    "color": "Color",
    "quantity": "SafetyStockLevel",
    "unit_price": "ListPrice"
})
sdm['Production_Product'] = pd.concat([sdm['Production_Product'], sdm['Products']], ignore_index=True)
sdm['Production_Product'] = pd.concat([sdm['Production_Product'], sdm['Product']], ignore_index=True)
sdm['Production_Product']['Weight'] = sdm['Production_Product']['Weight'].replace('', np.nan)
sdm['Production_Product']['Weight'] = sdm['Production_Product']['Weight'].astype(float)
del sdm['Products']
del sdm['Product']

sdm['Order_Details'] = sdm['Order_Details'].rename(columns={
    "OrderID": "SalesOrderID",
    "Discount": "UnitPriceDiscount",
    "Quantity": "quantity"
})
sdm['Sales_Order_Item'] = sdm['Sales_Order_Item'].rename(columns={
    "id": "SalesOrderID",
    "prod_id": "ProductID",
})
sdm['Sales_SalesOrderDetail'] = pd.concat([sdm['Sales_SalesOrderDetail'], sdm['Sales_Order_Item']], ignore_index=True)
sdm['Sales_SalesOrderDetail'] = pd.concat([sdm['Sales_SalesOrderDetail'], sdm['Order_Details']], ignore_index=True)
del sdm['Order_Details']
del sdm['Sales_Order_Item']


sdm['Customer'] = sdm['Customer'].rename(columns={"state": "state_id"})
sdm['Customers'] = sdm['Customers'].rename(columns={
    "CustomerID": "id",
    "CompanyName": "company_name",
    "Address": "address",
    "City": "city",
    "PostalCode": "zip",
    "Phone": "phone"
})
sdm['Customer'] = pd.concat([sdm['Customer'], sdm['Customers']], ignore_index=True)
del sdm['Customers']

sdm['Sales_Order'] = sdm['Sales_Order'].rename(columns={
    "id": "SalesOrderID",
    "cust_id": "CustomerID",
    "order_date": "OrderDate",
    "sales_rep": "SalesPersonID",
    "Region": "ShipRegion"
})
sdm['Orders'] = sdm['Orders'].rename(columns={
    "OrderID": "SalesOrderID",
    "order_date": "OrderDate",
    "EmployeeID": "SalesPersonID",
    "RequiredDate": "DueDate",
    "ShippedDate": "ShipDate"
})

sdm['Sales_SalesOrderHeader'] = sdm['Sales_SalesOrderHeader'].rename(columns={"CustomerID": "SalesCustomerID"})
sdm['Sales_SalesOrderHeader'] = pd.concat([sdm['Sales_SalesOrderHeader'], sdm['Sales_Order']], ignore_index=True)
sdm['Sales_SalesOrderHeader'] = pd.concat([sdm['Sales_SalesOrderHeader'], sdm['Orders']], ignore_index=True)
del sdm['Sales_Order']
del sdm['Orders']

sdm['Employees'] = sdm['Employees'].rename(columns={
    "EmployeeID": "emp_id",
    "LastName": "emp_lname",
    "FirstName": "emp_fname",
    "Address": "street",
    "City": "city",
    "Region": "state",
    "PostalCode": "zip_code",
    "HomePhone": "phone",
    "BirthDate": "birth_date",
    "HireDate": "start_date"
})
sdm['Employee'] = pd.concat([sdm['Employee'], sdm['Employees']], ignore_index=True)
del sdm['Employees']

sdm['Department'] = sdm['Department'].rename(columns={
    "dept_id": "DepartmentID",
    "dept_name": "Name",
})
sdm['HumanResources_Department'] = pd.concat([sdm['HumanResources_Department'], sdm['Department']], ignore_index=True)
sdm['Department'] = sdm['HumanResources_Department']
del sdm['HumanResources_Department']

sdm['Purchasing_PurchaseOrderDetail']['ReceivedQty'] = sdm['Purchasing_PurchaseOrderDetail']['ReceivedQty'].astype(float)
sdm['Purchasing_PurchaseOrderDetail']['RejectedQty'] = sdm['Purchasing_PurchaseOrderDetail']['RejectedQty'].astype(float)
sdm['Purchasing_PurchaseOrderDetail']['StockedQty'] = sdm['Purchasing_PurchaseOrderDetail']['StockedQty'].astype(float)

dict_order = [
    "State",
    "Region",
    "Department",
    "Employee",
    "Bonus",
    "Territories",
    "EmployeeTerritories",
    "Customer",
    "CustomerDemographics",
    "CustomerCustomerDemo",
    "Shippers",
    "Sales_SalesTerritory",
    "Person_Person",
    "Person_Address",
    "HumanResources_Employee",
    "Sales_Store",
    "Sales_Customer",
    "Production_ProductCategory",
    "Suppliers",
    "Production_Product",
    "Production_BillOfMaterials",
    "Purchasing_Vendor",
    "Purchasing_PurchaseOrderHeader",
    "Purchasing_PurchaseOrderDetail",
    "Sales_SalesOrderHeader",
    "Sales_SalesOrderDetail"

]               #Hierin komt dictionary order te staan.
                #Dit moet omdat de volgorde van welke tables worden gestuurd naar database belangrijk is.

sdm = {k: sdm[k] for k in dict_order if k in sdm} #Verandert de volgorde van sdm-Dictionary.

SDM in SSMS vullen:

In [90]:
import pandas as pd
import pyodbc

def format_value(value, column_name, table_name):
    reserved_keywords = ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'group', 'name', 'state']

    if column_name.lower() in reserved_keywords:
        column_name = f"[{column_name}]"

    if pd.isna(value):
        return "NULL"

    if isinstance(value, bool):
        return "1" if value else "0"

    if column_name.lower() in ['phone', 'zip_code']:
        return f"'{str(value)}'"
    
    if column_name.lower() == "customerid" and table_name.lower() != "Sales_Customer":
        return f"'{str(value)}'"
    
    elif isinstance(value, str):
        if value == "":
            return "NULL"
        return f"'{value.replace("'", "''")}'"
    
    elif isinstance(value, pd.Timestamp):
        return f"'{value.strftime('%Y-%m-%d')}'"
    
    elif isinstance(value, bytes):
        hex_value = value.hex()
        return f"0x{hex_value}"
    
    elif isinstance(value, (int, float)):
        return str(value)
    
    return f"'{str(value)}'"

def format_table_name(table_name):
    reserved_keywords = ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'state']

    if table_name.lower() in reserved_keywords:
        return f"[{table_name}]"
    
    return table_name 

export_cursor = export_conn.cursor()

for table_name, df in sdm.items():

    error_count = 0
    total_count = 0

    try:
        print(f"Loading table {table_name}:", end=" ")

        formatted_table_name = format_table_name(table_name)

        for index, row in df.iterrows():
            total_count += 1

            columns = df.columns.tolist()

            if 'rowguid' in columns:
                columns.remove('rowguid')

            formatted_columns = [
                f"[{col}]" if col.lower() in ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'group', 'name'] else col
                for col in columns
            ]

            values = []
            for col in columns:
                value = row[col]
                values.append(format_value(value, col, table_name))

            column_names = ", ".join(formatted_columns)
            value_string = ", ".join(values)

            query = f"INSERT INTO {formatted_table_name} ({column_names}) VALUES ({value_string})"
            try:
                export_cursor.execute(query)
            except pyodbc.Error as e:
                error_count += 1
                continue
    
        export_conn.commit()
        

        passed_count = total_count - error_count

        if error_count == 0:
            print(f"100% ✓")
        else:
            print(f"{passed_count}/{total_count} rows ✓")

    except pyodbc.Error as e:
        print("✗ ERROR")
        print(f"{query}")
        print(f"Error message: {e}")
        print("-" * 80)

export_cursor.close()


Loading table State: 100% ✓
Loading table Region: 100% ✓
Loading table Department: 100% ✓
Loading table Employee: 100% ✓
Loading table Bonus: 100% ✓
Loading table Territories: 100% ✓
Loading table EmployeeTerritories: 100% ✓
Loading table Customer: 100% ✓
Loading table CustomerDemographics: 100% ✓
Loading table CustomerCustomerDemo: 100% ✓
Loading table Shippers: 100% ✓
Loading table Sales_SalesTerritory: 100% ✓
Loading table Person_Person: 100% ✓
Loading table Person_Address: 18798/19614 rows ✓
Loading table HumanResources_Employee: 100% ✓
Loading table Sales_Store: 100% ✓
Loading table Sales_Customer: 100% ✓
Loading table Production_ProductCategory: 100% ✓
Loading table Suppliers: 100% ✓
Loading table Production_Product: 512/591 rows ✓
Loading table Production_BillOfMaterials: 100% ✓
Loading table Purchasing_Vendor: 100% ✓
Loading table Purchasing_PurchaseOrderHeader: 100% ✓
Loading table Purchasing_PurchaseOrderDetail: 100% ✓
Loading table Sales_SalesOrderHeader: 28489/32945 rows ✓
